# **Détection de Langue: Démo**

In [23]:
# Imports
from pathlib import Path
import sys
import json
import torch
import torch.nn.functional as F
import sentencepiece as spm


In [24]:
PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

In [34]:
from src.model import BiLSTMSubword
from src.predict import encode_text_sp, predict_language
import src

In [35]:
import importlib
import src

importlib.reload(src)
importlib.reload(src.predict)
importlib.reload(src.preprocess)
# importlib.reload(src.data_loader)
importlib.reload(src.model)

<module 'src.model' from '/workspaces/language-detection-nlp/src/model.py'>

In [15]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CKPT_PATH = PROJECT_ROOT / "models" / "bilstm_subword_final.pt"
SPM_PATH  = PROJECT_ROOT / "notebooks" / "spm_lang_detector.model"
LABELS_PATH = PROJECT_ROOT / "models" / "labels.json"

print("Device:", DEVICE)
print("CKPT:", CKPT_PATH)
print("SPM :", SPM_PATH)
print("LBL :", LABELS_PATH)

Device: cpu
CKPT: /workspaces/language-detection-nlp/models/bilstm_subword_final.pt
SPM : /workspaces/language-detection-nlp/notebooks/spm_lang_detector.model
LBL : /workspaces/language-detection-nlp/models/labels.json


In [12]:
# Chargeons les labels 'langue' - 'id'
with open(LABELS_PATH, "r", encoding="utf-8") as f:
    labels = json.load(f)

id2label = {i: labels[i] for i in range(len(labels))}

print("Nombre de labels:", len(labels))
print("Aperçu:", labels[:5])

Nombre de labels: 17
Aperçu: ['Arabic', 'Danish', 'Dutch', 'English', 'French']


In [13]:
# Chargeons SentencePiece
sp = spm.SentencePieceProcessor()
sp.load(str(SPM_PATH))

print("SentencePiece vocab_size:", sp.get_piece_size())

SentencePiece vocab_size: 8000


In [14]:
# Charger le `checkpoint` et reconstruire le modèle

checkpoint = torch.load(CKPT_PATH, map_location=DEVICE, weights_only=False)

model = BiLSTMSubword(
    vocab_size=checkpoint["vocab_size"],
    embedding_dim=checkpoint["embedding_dim"],
    hidden_dim=checkpoint["hidden_dim"],
    num_classes=checkpoint["num_classes"],
    n_layers=checkpoint["n_layers"],
    dropout=checkpoint["dropout"],
)

model.load_state_dict(checkpoint["model_state_dict"])
model = model.to(DEVICE)
model.eval()

print("num_classes:", checkpoint["num_classes"])
print("params:", sum(p.numel() for p in model.parameters()))


num_classes: 17
params: 17818641


# **Démo rapide**

In [42]:
tests =  [

"Ce projet de recherche vise à développer un système automatique capable de détecter la langue d’un texte donné avec une grande précision. L’objectif est d’analyser différentes approches d’apprentissage profond afin d’identifier la méthode la plus robuste face à des textes variés, qu’ils soient courts, longs, techniques ou conversationnels. L’évaluation du modèle repose sur plusieurs métriques telles que l’accuracy, le rappel et le score F1, tout en observant attentivement les erreurs entre langues proches partageant des caractéristiques lexicales similaires. Cette analyse approfondie permet d’améliorer progressivement les performances du système.", 

"This research initiative focuses on designing an advanced language detection system using deep learning techniques. The goal is to create a model capable of identifying the language of an input text with high reliability, even when the sentences come from different domains or writing styles. The evaluation process includes analyzing confusion patterns between similar languages and studying how subword tokenization improves discrimination. By testing the model on external corpora, we aim to ensure that it generalizes effectively beyond the original training dataset.", 

"Este proyecto tiene como finalidad desarrollar un sistema inteligente capaz de identificar automáticamente el idioma de un texto determinado. Para lograrlo, se entrenan modelos de aprendizaje automático con datos multilingües y se evalúan sus resultados mediante métricas detalladas. La capacidad de generalización del modelo se analiza utilizando textos externos, lo que permite observar posibles confusiones entre idiomas que comparten estructuras gramaticales similares. Este enfoque garantiza una evaluación más realista del rendimiento del sistema.", 

"Dieses Projekt beschäftigt sich mit der Entwicklung eines Systems zur automatischen Spracherkennung auf der Grundlage moderner Methoden des maschinellen Lernens. Das Modell wird mit einem umfangreichen mehrsprachigen Datensatz trainiert und anschließend auf unabhängigen Texten getestet, um seine Generalisierungsfähigkeit zu überprüfen. Besonders wichtig ist die Analyse von Fehlern zwischen Sprachen mit ähnlicher Struktur, da diese häufig verwechselt werden. Durch eine sorgfältige Auswertung der Ergebnisse kann die Leistung des Systems kontinuierlich verbessert werden.", 

"يهدف هذا المشروع البحثي إلى تطوير نظام ذكي قادر على التعرف التلقائي على لغة النصوص المكتوبة بدقة عالية. يتم تدريب النموذج باستخدام بيانات متعددة اللغات ثم يتم تقييم أدائه على نصوص جديدة للتأكد من قدرته على التعميم خارج مجموعة التدريب الأصلية. كما يتم تحليل الأخطاء التي تحدث بين اللغات المتشابهة في البنية أو المفردات، مما يساعد على تحسين النموذج وجعله أكثر استقرارًا في البيئات الواقعية." 
]

for t in tests:
    res = predict_language(t, model, sp, id2label, DEVICE, top_k=3)
    print("\nTEXT:", t)
    print("TOP-3:", res)


TEXT: Ce projet de recherche vise à développer un système automatique capable de détecter la langue d’un texte donné avec une grande précision. L’objectif est d’analyser différentes approches d’apprentissage profond afin d’identifier la méthode la plus robuste face à des textes variés, qu’ils soient courts, longs, techniques ou conversationnels. L’évaluation du modèle repose sur plusieurs métriques telles que l’accuracy, le rappel et le score F1, tout en observant attentivement les erreurs entre langues proches partageant des caractéristiques lexicales similaires. Cette analyse approfondie permet d’améliorer progressivement les performances du système.
TOP-3: [('French', 0.5606784224510193), ('Portugeese', 0.41460126638412476), ('Spanish', 0.011799817904829979)]

TEXT: This research initiative focuses on designing an advanced language detection system using deep learning techniques. The goal is to create a model capable of identifying the language of an input text with high reliabilit

### **Vérifions que le pipeline d’inférence est identique au training (diagnostic rapide)**

In [44]:
def debug_sp(text, sp):
    pieces = sp.encode(text, out_type=str)
    ids = sp.encode(text, out_type=int)
    unk_id = 1  # <unk>
    unk_count = sum(1 for i in ids if i == unk_id)
    return {
        "text": text,
        "n_tokens": len(ids),
        "unk_count": unk_count,
        "unk_ratio": unk_count / max(1, len(ids)),
        "first_pieces": pieces[:30],
        "first_ids": ids[:30],
    }

print(debug_sp("welcome everyone", sp))
print(debug_sp("nature in the broadest sense is the natural physical material world", sp))


{'text': 'welcome everyone', 'n_tokens': 6, 'unk_count': 0, 'unk_ratio': 0.0, 'first_pieces': ['▁', 'wel', 'co', 'me', '▁every', 'one'], 'first_ids': [4, 2508, 226, 221, 3959, 1091]}
{'text': 'nature in the broadest sense is the natural physical material world', 'n_tokens': 14, 'unk_count': 0, 'unk_ratio': 0.0, 'first_pieces': ['▁nature', '▁in', '▁the', '▁broad', 'est', '▁sens', 'e', '▁is', '▁the', '▁natural', '▁physic', 'al', '▁material', '▁world'], 'first_ids': [159, 17, 14, 5682, 855, 865, 6, 51, 14, 1121, 3840, 60, 2072, 1717]}


<u>**Observations:**</u>
Ce qui se passe réellement <br />
Le modèle est :
- Très performant sur le test Kaggle (≈ 95%). Mais mal calibré sur des phrases courtes, génériques et hors domaine Wikipédia
“Welcome everyone” est :
- Très court
- Très neutre
- Lexicalement proche de plusieurs langues latines
- Peu informatif statistiquement
- Le modèle choisit donc une langue latine fréquente (Italian ici).

C’est un comportement normal en production pour un modèle entraîné sur textes longs.

**Conclusion technique**
Ce n’est pas un bug.<br />
C’est un problème de :
- Distribution shift (Wikipédia vs phrase libre)
- Manque de contexte
- Ambiguïté linguistique courte


**Conclusion complémentaire**

La performance du modèle est directement corrélée à la quantité d’information contextuelle disponible.  
Plus le texte est long et structuré, plus la prédiction est stable et fiable.
